#Imports

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Colab specific tools
from google.colab.patches import cv2_imshow


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Standard library imports
import sys
import os
import io
import traceback

# Data handling
import pandas as pd
import numpy as np

# Image processing and computer vision
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

# For combinations
from itertools import combinations

# Adjust the Python path to include a specific directory
sys.path.append('/content/drive/MyDrive/colabpackages')

# YOLO for object detection and Roboflow for dataset handling
from ultralytics import YOLO
from roboflow import Roboflow

# Visualization tools
import matplotlib.pyplot as plt

# Gradio for creating interfaces
!pip install gradio
import gradio as gr


#Models

In [ ]:

# Load trained models
fill_model = load_model('/content/drive/MyDrive/Models/Characteristics/17052024/fill_model_model.h5')
shape_model = load_model('/content/drive/MyDrive/Models/Characteristics/17052024/shape_model_model.h5')

# Load YOLO models for shape and card detection
shape_detection_model = YOLO('/content/drive/MyDrive/Models/Shape/15052024/best.pt')
shape_detection_model.yaml = '/content/drive/MyDrive/Models/Shape/15052024/data.yaml'
shape_detection_model.conf = 0.5

card_detection_model = YOLO('/content/drive/MyDrive/Models/Card/16042024/best.pt')
card_detection_model.yaml= '/content/drive/MyDrive/Models/Card/16042024/data.yaml'
card_detection_model.conf = 0.5


#Predictions


In [ ]:
def predict_color(shape_image):
    """
    Predict the color of the given shape image using HSV color space.

    Parameters:
    shape_image (numpy array): The image of the shape.

    Returns:
    str: The predicted color.
    """
    hsv_image = cv2.cvtColor(shape_image, cv2.COLOR_BGR2HSV)

    # Define color ranges in HSV
    color_ranges = {
        'green': [(40, 50, 50), (80, 255, 255)],
        'red': [(120, 50, 50), (160, 255, 255)],
        'purple': [(0, 50, 50), (10, 255, 255), (170, 50, 50), (180, 255, 255)]
    }

    color_counts = {}
    for color, ranges in color_ranges.items():
        if color == 'purple':
            lower1, upper1, lower2, upper2 = ranges
            mask1 = cv2.inRange(hsv_image, np.array(lower1), np.array(upper1))
            mask2 = cv2.inRange(hsv_image, np.array(lower2), np.array(upper2))
            mask = cv2.bitwise_or(mask1, mask2)
        else:
            lower, upper = ranges
            mask = cv2.inRange(hsv_image, np.array(lower), np.array(upper))
        color_counts[color] = cv2.countNonZero(mask)

    predicted_color = max(color_counts, key=color_counts.get)
    return predicted_color


In [ ]:
def detect_cards(board_image, card_detection_model):
    """
    Detect cards on the board using the card detection model.

    Parameters:
    board_image (numpy array): The image of the board.
    card_detection_model (YOLO): The YOLO model for card detection.

    Returns:
    list: A list of tuples containing card images and their bounding boxes.
    """
    try:
        card_results = card_detection_model(board_image)
        card_boxes = card_results[0].boxes.xyxy.cpu().numpy().astype(int)

        cards = []
        for box in card_boxes:
            x1, y1, x2, y2 = box
            card_image = board_image[y1:y2, x1:x2]
            cards.append((card_image, box))

        return cards
    except Exception as e:
        print(f"Error in detect_cards: {str(e)}")
        return []


In [ ]:
def predict_card_features(card_image, shape_detection_model, fill_model, shape_model):
    """
    Predict the features of the card (color, fill, shape) using the provided models.

    Parameters:
    card_image (numpy array): The image of the card.
    shape_detection_model (YOLO): The YOLO model for shape detection.
    fill_model (Model): The Keras model for fill prediction.
    shape_model (Model): The Keras model for shape prediction.

    Returns:
    dict: A dictionary containing the predicted card characteristics.
    """
    try:
        shape_results = shape_detection_model(card_image)
        card_height, card_width = card_image.shape[:2]
        card_area = card_width * card_height

        filtered_boxes = []
        for box in shape_results[0].boxes.xyxy.cpu().numpy():
            x1, y1, x2, y2 = box.astype(int)
            shape_width = x2 - x1
            shape_height = y2 - y1
            shape_area = shape_width * shape_height

            if shape_area > 0.03 * card_area:
                filtered_boxes.append(box)

        count = min(len(filtered_boxes), 3)

        if count > 0:
            color_labels = []
            fill_labels = []
            shape_labels = []

            for shape_box in filtered_boxes:
                shape_box = shape_box.astype(int)
                shape_image = card_image[shape_box[1]:shape_box[3], shape_box[0]:shape_box[2]]

                fill_input_shape = fill_model.input_shape[1:3]
                shape_input_shape = shape_model.input_shape[1:3]

                fill_shape_image = cv2.resize(shape_image, fill_input_shape) / 255.0
                shape_shape_image = cv2.resize(shape_image, shape_input_shape) / 255.0

                fill_pred = fill_model.predict(np.expand_dims(fill_shape_image, axis=0))
                shape_pred = shape_model.predict(np.expand_dims(shape_shape_image, axis=0))

                color_labels.append(predict_color(shape_image))
                fill_labels.append(['empty', 'full', 'striped'][np.argmax(fill_pred)])
                shape_labels.append(['diamond', 'oval', 'squiggle'][np.argmax(shape_pred)])

            color_label = max(set(color_labels), key=color_labels.count)
            fill_label = max(set(fill_labels), key=fill_labels.count)
            shape_label = max(set(shape_labels), key=shape_labels.count)
        else:
            color_label = fill_label = shape_label = 'unknown'

        card_prediction = {
            'count': count,
            'color': color_label,
            'fill': fill_label,
            'shape': shape_label
        }

        return card_prediction
    except Exception as e:
        print(f"Error in predict_card_features: {str(e)}")
        return {'count': 'unknown', 'color': 'unknown', 'fill': 'unknown', 'shape': 'unknown'}


In [ ]:
def classify_cards(board_image):
    """
    Classify the cards on the board image.

    Parameters:
    board_image (Image): The image of the board.

    Returns:
    DataFrame: A pandas DataFrame containing the card data.
    """
    try:
        board_image_cv = cv2.cvtColor(np.array(board_image), cv2.COLOR_RGB2BGR)
        cards = detect_cards(board_image_cv, card_detection_model)
        card_data = []

        for card_image, box in cards:
            card_features = predict_card_features(card_image, shape_detection_model, fill_model, shape_model)
            card_data.append({
                "Count": card_features['count'],
                "Color": card_features['color'],
                "Fill": card_features['fill'],
                "Shape": card_features['shape'],
                "Coordinates": f"{box[0]}, {box[1]}, {box[2]}, {box[3]}"
            })

        return pd.DataFrame(card_data)
    except Exception as e:
        print(f"Error in classify_cards: {str(e)}")
        return pd.DataFrame()


In [ ]:
def is_set(cards):
    """
    Check if the given cards form a valid set.

    Parameters:
    cards (list): List of cards to be checked.

    Returns:
    bool: True if the cards form a valid set, otherwise False.
    """
    for card in cards:
        if 'unknown' in card.tolist():
            return False

    for feature in ['Count', 'Color', 'Fill', 'Shape']:
        if len(set(card[feature] for card in cards)) not in [1, 3]:
            return False
    return True


In [ ]:
def find_sets(card_df):
    """
    Find all possible sets in the given DataFrame of card data.

    Parameters:
    card_df (DataFrame): DataFrame containing the card data.

    Returns:
    list: A list of sets found.
    """
    sets_found = []
    card_combinations = combinations(card_df.iterrows(), 3)

    for combo in card_combinations:
        cards = [card[1] for card in combo]
        card_indices = [card[0] for card in combo]

        if is_set(cards):
            set_info = {
                'set_indices': card_indices,
                'cards': [{feature: card[feature] for feature in ['Count', 'Color', 'Fill', 'Shape', 'Coordinates']} for card in cards]
            }
            sets_found.append(set_info)

    # Check for sets with shared cards
    sets_to_remove = []
    for i in range(len(sets_found)):
        for j in range(i + 1, len(sets_found)):
            shared_cards = set(sets_found[i]['set_indices']) & set(sets_found[j]['set_indices'])
            if len(shared_cards) > 1:
                sets_to_remove.append(sets_found[j])

    # Remove sets with more than one shared card
    sets_found = [set_info for set_info in sets_found if set_info not in sets_to_remove]

    return sets_found


In [ ]:
def classify_and_find_sets(board_image):
    """
    Classify cards on the board image and find all possible sets.

    Parameters:
    board_image (Image): The image of the board.

    Returns:
    list: A list of sets found.
    """
    card_df = classify_cards(board_image)
    sets_found = find_sets(card_df)
    return sets_found


In [ ]:
def draw_sets(board_image, sets_info):
    """
    Draw bounding boxes around detected sets on the board image.

    Parameters:
    board_image (numpy array): The image of the board.
    sets_info (list): A list of sets found.

    Returns:
    numpy array: The board image with bounding boxes drawn around sets.
    """
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
    image_height, image_width = board_image.shape[:2]

    # Scaling factors based on the image size
    scale_factor = min(image_width, image_height) / 500
    base_thickness = int(2 * scale_factor)
    base_expansion = int(2 * scale_factor)  # Reduced base_expansion
    font_scale = 0.5 * scale_factor
    font_thickness = int(1 * scale_factor)

    for index, set_info in enumerate(sets_info):
        color = colors[index % len(colors)]
        thickness = base_thickness + int(0.5 * index)
        expansion = base_expansion + int(5 * index)  # Increased expansion factor

        for card in set_info['cards']:
            coordinates = list(map(int, card['Coordinates'].split(',')))
            x1, y1, x2, y2 = coordinates

            x1_expanded = max(0, x1 - expansion)
            y1_expanded = max(0, y1 - expansion)
            x2_expanded = min(board_image.shape[1], x2 + expansion)
            y2_expanded = min(board_image.shape[0], y2 + expansion)

            cv2.rectangle(board_image, (x1_expanded, y1_expanded), (x2_expanded, y2_expanded), color, thickness)

            if card == set_info['cards'][0]:
                cv2.putText(board_image, f"Set {index + 1}", (x1_expanded, y1_expanded - int(10 * scale_factor)),
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, font_thickness)

    return board_image

In [ ]:
def detect_and_display_sets_interface(input_image):
    """
    Detect and display sets on the input image using the Gradio interface.

    Parameters:
    input_image (Image): The input image of the board.

    Returns:
    tuple: The image with sets highlighted and a success message.
    """
    try:
        input_image_cv = cv2.cvtColor(np.array(input_image), cv2.COLOR_RGB2BGR)
        sets_info = classify_and_find_sets(input_image_cv)
        board_image_with_sets = draw_sets(input_image_cv, sets_info)

        return Image.fromarray(cv2.cvtColor(board_image_with_sets, cv2.COLOR_BGR2RGB))

    except Exception as e:
        error_details = "Error: " + str(e) + "\n" + traceback.format_exc()
        return Image.fromarray(np.zeros((100, 100, 3), dtype=np.uint8))

#Gradio

In [ ]:
# Gradio

# Create the Gradio interface
iface = gr.Interface(
    fn=detect_and_display_sets_interface,
    inputs=gr.components.Image(type="pil"),
    outputs=gr.components.Image(type="pil"),
    title="Set Game Detector",
    description="Upload an image of a Set game board, and if there are sets found in the image the interface will highlight them"
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://647dab153716d8b611.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Launch the interface
iface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://647dab153716d8b611.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
